In [1]:
# Import packages
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
import math
%pylab inline
import matplotlib.pyplot as plt
from nms import nms
import glob as glob
# This is needed to display the images.
%matplotlib inline

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Import utilites
from utils import label_map_util    #, get_label_map_dict
from utils import visualization_utils as vis_util

#### Helper Function
def createDirectory(path):
    if not os.path.isdir(path):
        os.makedirs(path)
        print ("Creating {}".format(path))
    return path

print(tf.__version__)

Populating the interactive namespace from numpy and matplotlib
1.15.0


### Template Matching

In [2]:
class TM_PinPredction():
    
    #########################################
    # Member Data structures
    #########################################    
    class Filter():
        '''
        Failed if the prediction is more than -+25% offset from the template center
        '''
        def assert_template_ycenter_inrange(self, ymin, ymax, y1, h):
            cy = y1 + h/2
            if (cy<(ymin + h/4) or cy>(ymax - h/4)):
                return False
            return True
                
    #########################################
    # Member Methods
    #########################################       
    def __init__(self, dir_path, out_dir):
        #Path to img and txt dir
        self.dir_path=dir_path 
        self.out_dir=out_dir
        self.ai_score_threshold=0.9
        self.template_threshold=0.6
    

    def generate_txt_file_list(self):
        txt_files = glob.glob(os.path.join(self.dir_path, "*.txt"))
        
        #Validation
        for e in txt_files:
            if "classes.txt" in e:
                txt_files.remove(e)
                continue
            if not (os.path.isfile(e.replace(".txt", ".jpg"))):
                txt_files.remove(e)
                print("Not Found: ", e)
                
        #Return
        return txt_files
    
    def generate_prediction_using_template_matching(self, img_fname, txt_fname):
        
        #Read
        txt_df=pd.read_csv(txt_fname, 
                    names=['x1', 'y1', 'x2', 'y2', 'class', 'score'],  
                    dtype={'x1': np.int, 'y1': np.int, 'x2': np.int, 'y2': np.int, 'class':str, 'score': np.float64} )
        
        img = cv2.imread(img_fname, 0)
        name = os.path.basename(img_fname)
        
        #Output
        txt_out_fpath=os.path.join(self.out_dir, name.replace(".jpg", ".txt"))
        
        #Limit
        ymax = txt_df.max(axis = 0).to_dict()['y2']
        ymin = txt_df.min(axis = 0).to_dict()['y1'] 
        
        # GT
        gt_img = img.copy()
        gt_dir = createDirectory(os.path.join(self.out_dir,"GT"))
        for i, r_row in txt_df.iterrows():
            r = r_row.to_dict()
            cv2.rectangle(gt_img, (r['x1'], r['y1']), (r['x2'], r['y2']), (255,0,255), 1)
        cv2.imwrite(os.path.join(gt_dir, name), gt_img)

        # Prediction
        boxes =[]
        scores =[]
        ai_df = txt_df.copy()
        for i, row in txt_df.iterrows():
            e = row.to_dict()
            if e['score']<self.ai_score_threshold:
                continue

            template = img.copy()[ e['y1']:e['y2'], e['x1']:e['x2']]
            h, w =template.shape
            res = cv2.matchTemplate(img,template,cv2.TM_CCOEFF_NORMED)
            threshold = self.template_threshold
            loc = np.where( res >= threshold)
            temp =img.copy()
            cboxes=[]
            cscores=[]
            for pt in zip(*loc[::-1]):
                if not (self.filters.assert_template_ycenter_inrange(ymin, ymax, pt[1],h)):
                    continue
                cboxes.append((pt[0],pt[1],w,h))
                score = res[pt[1], pt[0]]
                cscores.append(score)

            #Local NMS
            for ibox in nms.boxes(cboxes, cscores): 
                boxes.append(cboxes[ibox])
                scores.append(cscores[ibox])
        
        #Global Nms
        test = img.copy()
        with open(txt_out_fpath, 'w') as f:
            for idx in nms.boxes(boxes, scores):
                x1, y1, w, h=boxes[idx]
                score = scores[idx]
                cv2.rectangle(test, (x1, y1), (x1 + w, y1 + h), (255,255,255), 1)
                f.write("{},{},{},{},{},{}\n".format(x1, y1, x1+w, y1+h, 'PIN', score))
        
        # Save
        cv2.imwrite(txt_out_fpath.replace(".txt",".jpg"), test)

                
        #Comparative Image
        cmp_dir = createDirectory(os.path.join(self.out_dir,"CMP"))
        cmp_img = np.zeros(shape=[gt_img.shape[0]*2, gt_img.shape[1]], dtype=np.uint8)
        cmp_img[0*gt_img.shape[0]:(0+1)*gt_img.shape[0], :gt_img.shape[1]] = gt_img.copy()
        cmp_img[1*test.shape[0]:(1+1)*test.shape[0], :test.shape[1]] = test.copy()
        cv2.imwrite(os.path.join(cmp_dir, name), cmp_img)
        
        
    def predict_multiple_image(self):
        self.filters = self.Filter()
        txt_files = self.generate_txt_file_list()
        for txt_file in txt_files:
            img_fname = txt_file.replace(".txt", ".jpg")    
            self.generate_prediction_using_template_matching(img_fname, txt_file)
            
            

    def predict_single_image(self,img_fname, txt_file):
        self.filters = self.Filter()
        self.generate_prediction_using_template_matching(img_fname, txt_file)
        

In [3]:
in_dir = "D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/scripts/Samples/ML_Sample-5/Input"
out_dir = createDirectory ("D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/scripts/Samples/ML_Sample-5/Output")
# tm_model = TM_PinPredction(in_dir, out_dir)
# tm_model.predict_multiple_image()

_______________________
### Ai outlier detector

The target is to take all the boxes predicted by Ai and compare between each of them then pass flag the ones that are not a good fit

**Steps**
1. Read ai prediction csv file to a variable.
2. Crop out all of the predicted roi

In [2]:
class AI_Outlier_Detector():

    #########################################
    # Member Data structures
    #########################################
    class SideWiseFilter():
            '''
            Description
                - Created to hold all the condition checks 
                - Ideally the member function should return true or false
            '''
            ################
            # Member Methods
            ################
            def is_sideoffset_tl_section_valid(self, img):
                    h, w = img.shape
                    roi_x1 = int(w * 0.75)
                    roi_y2 = int(h *0.5)
                    roi = img.copy()[ :roi_y2, roi_x1:]
                    _minVal, _maxVal, minLoc, maxLoc = cv2.minMaxLoc(roi, None)
                    return _maxVal > 0
            def is_sideoffset_tr_section_valid(self, img):
                    h, w = img.shape
                    roi_x2 = int(w * 0.25)
                    roi_y2 = int(h *0.5)
                    roi = img.copy()[ :roi_y2, :roi_x2]
                    _minVal, _maxVal, minLoc, maxLoc = cv2.minMaxLoc(roi, None)
                    return _maxVal > 0
            def is_sideoffset_l_section_valid(self, img):
                    h, w = img.shape
                    roi_x1 = int(w * 0.85)
                    roi_y1 = int(h *0.5)
                    roi = img.copy()[ :, roi_x1:]
                    _minVal, _maxVal, minLoc, maxLoc = cv2.minMaxLoc(roi, None)
                    return _maxVal > 0
            def is_sideoffset_r_section_valid(self, img):
                    h, w = img.shape
                    roi_x1 = int(w * 0.85)
                    roi_y1 = int(h *0.5)
                    roi = img.copy()[ :, roi_x1:]
                    _minVal, _maxVal, minLoc, maxLoc = cv2.minMaxLoc(roi, None)
                    return _maxVal > 0
    
    
    class ExtendWiseFilter():
            '''
            Description
                - Created to hold all the condition checks 
                - Ideally the member function should return true or false
            '''
            ################
            # Member Methods
            ################
            def get_empty_leftpad(self, img):
                h, w = img.shape
                np.sum(img, axis=0)

                
    #########################################
    # Member Methods
    #########################################         
    def __init__(self, dir_path, out_dir):
        #Path to img and txt dir
        self.dir_path=dir_path 
        self.out_dir=out_dir

    def generate_txt_file_list(self):
        txt_files = glob.glob(os.path.join(self.dir_path, "*.txt"))

        #Validation
        for e in txt_files:
            if "classes.txt" in e:
                txt_files.remove(e)
                continue
            if not (os.path.isfile(e.replace(".txt", ".jpg"))):
                txt_files.remove(e)
                print("Not Found: ", e)

        #Return
        return txt_files


    def read_data (self, img_fname, txt_fname):
        txt_df=pd.read_csv(txt_fname, 
                    names=['x1', 'y1', 'x2', 'y2', 'class', 'score'],  
                    dtype={'x1': np.int, 'y1': np.int, 'x2': np.int, 'y2': np.int, 'class':str, 'score': np.float64} )

        img = cv2.imread(img_fname, 0)
        return  img, txt_df


    def  gen_cropped_imgarray(self, in_image, txt_df, img_array):
        img=in_image.copy()
        for i, row in txt_df.iterrows():
            r = row.to_dict()
            cropped_img = img[r['y1']:r['y2'], r['x1']:r['x2']]
            img_array.append(cropped_img)

    def print_img_array(self, img_array, xblock=4, titles=[]):
        fig = plt.figure()
        xblock = len(img_array) if xblock> len(img_array) else xblock
        yblock=math.ceil(len(img_array)/xblock)
        for i, img in enumerate(img_array):
            subplot = fig.add_subplot(yblock, xblock, i +1)
            if (len(titles)>0): subplot.set_title(titles[i])
            plt.imshow(img, cmap='gray')
            
        

    '''
    Run
    '''
    def run_sidewise_pad_outlier_detector(self):
        plt.close('all') 
        txt_files = self.generate_txt_file_list()
        img_array=[]
        img_threshold_array=[]
        print(txt_files)
        for txt_file in txt_files:
            img_fname = txt_file.replace(".txt", ".jpg")   
            name = os.path.basename(img_fname)
            img, txt_df = self.read_data (img_fname, txt_file)
            self.gen_cropped_imgarray(img, txt_df, img_array)
            print(name)
        self.print_img_array(img_array, xblock=10)
        
        outlier_filter = self.Filter()
        status_titles=[]
        for img in img_array:
            
            # Otsu's thresholding after Gaussian filtering
            blur = cv2.GaussianBlur(img,(5,5),0)
            ret3,img = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            img_threshold_array.append(img)
            
            #Filter
            if not (outlier_filter.is_sideoffset_tl_section_valid(img) ):
                status_titles.append('FAIL')
            elif not ( outlier_filter.is_sideoffset_tr_section_valid(img)):
                status_titles.append('FAIL')
            elif not ( outlier_filter.is_sideoffset_r_section_valid(img)):
                status_titles.append('FAIL')
            else:
                status_titles.append('PASS')
        self.print_img_array(img_threshold_array, xblock=10, titles=status_titles)
        
    
    def run_extended_border_outlierdetector(self):
            plt.close('all') 
            txt_files = self.generate_txt_file_list()
            
            # Declaration
            img_array=[]
            img_array_extleft=[]
            img_array_extright=[]
            img_threshold_array=[]
            img_array_threshold_extleft=[]
            img_array_threshold_extright=[]
            
            
            
            for txt_file in txt_files:
                img_fname = txt_file.replace(".txt", ".jpg")   
                name = os.path.basename(img_fname)
                img, txt_df = self.read_data (img_fname, txt_file)
                
                #Cropped
                dx=5
                for i, row in txt_df.iterrows():
                    r = row.to_dict()
                    h, w = img.shape
                    print(r['x2'] - r['x1'])
                    if  (r['x2'] - r['x1']) >21:
                        print("skip")
                        continue
                    dx = int(0.1 *(r['x2'] - r['x1'])) + 1
                    print(dx)
                    cropped_img = img[r['y1']:r['y2'], r['x1']:r['x2']]
                    img_array.append(img.copy()[r['y1']:r['y2'], r['x1']:r['x2']])
                    img_array_extleft.append(img.copy()[r['y1']:r['y2'], r['x1']-dx:r['x2']])
                    img_array_extright.append(img.copy()[r['y1']:r['y2'], r['x1']:r['x2']+dx])
            
            # Thresholding
            def apply_thresholding(img_array, img_threshold_array):
                for img in img_array:
                    # Otsu's thresholding after Gaussian filtering
                    blur = cv2.GaussianBlur(img,(5,5),0)
                    ret3,img = cv2.threshold(blur,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
                    img_threshold_array.append(img)
            
            apply_thresholding(img_array, img_threshold_array)   
            apply_thresholding(img_array_extleft, img_array_threshold_extleft)
            apply_thresholding(img_array_extright, img_array_threshold_extright)     

            
            for i, img in enumerate(img_array):
                vsum = np.sum(img_array_threshold_extleft[i], axis=0)
                statuslf = np.sum(vsum[0:3])<1
                cv2.imwrite(os.path.join(self.out_dir, "img_array_threshold_extleft"+str(i)+"-"+str(statuslf)+".jpg"), img_array_threshold_extleft[i])
                
                vsum = np.sum(img_array_threshold_extright[i], axis=0)
                statusrt =  np.sum(vsum[-3:])<1 
                print(str(i),"Result",str(statusrt and statuslf), "\tL Pad:\t", statuslf,"\tR Pad:\t", statusrt)
                cv2.imwrite(os.path.join(self.out_dir, "img_array_threshold_extright"+str(i)+"-"+str(statusrt)+".jpg"), img_array_threshold_extright[i])
                
                
                cv2.imwrite(os.path.join(self.out_dir, "img_array"+str(i)+"-"+str(statusrt and statuslf)+".jpg"), img)
                cv2.imwrite(os.path.join(self.out_dir, "img_threshold_array"+str(i)+"-"+str(statusrt and statuslf)+".jpg"), img_threshold_array[i])


In [30]:
plt.close("all")
in_dir = "D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/scripts/Samples/Ai_Outlier-1/Input"
out_dir = createDirectory ("D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/scripts/Samples/Ai_Outlier-1/Output")
!rm "D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/scripts/Samples/Ai_Outlier-1/Output/*"
detection_model = AI_Outlier_Detector(in_dir, out_dir)
detection_model.run_extended_border_outlierdetector()


23
skip
22
skip
23
skip
20
3
20
3
20
3
20
3
20
3
20
3
20
3
20
3
20
3
0 Result True 	L Pad:	 True 	R Pad:	 True
1 Result False 	L Pad:	 True 	R Pad:	 False
2 Result True 	L Pad:	 True 	R Pad:	 True
3 Result True 	L Pad:	 True 	R Pad:	 True
4 Result True 	L Pad:	 True 	R Pad:	 True
5 Result True 	L Pad:	 True 	R Pad:	 True
6 Result True 	L Pad:	 True 	R Pad:	 True
7 Result False 	L Pad:	 True 	R Pad:	 False
8 Result True 	L Pad:	 True 	R Pad:	 True


rm: D:/FZ_WS/JyNB/TF_Research_Api_LD_2_0/research/object_detection/scripts/Samples/Ai_Outlier-1/Output/*: No such file or directory
